## 1. Setup

In [1]:
import sys
sys.path.append('../..')

In [2]:
import config
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import warnings

from neural_networks.seg_reg_net import SegRegNet
from neural_networks.net_utils import predict_density_maps_and_get_counts
from utils.data.data_generator import DataGenerator
from utils.evaluation.evaluation import evaluation_results_as_dict
from utils.evaluation.evaluation import evaluation_results_as_df
from utils.input_output.io import load_images_and_density_maps
from utils.input_output.io import read_json, write_json
from utils.input_output.io import load_gt_counts

Using TensorFlow backend.


In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

warnings.filterwarnings('ignore')

## 2. Load the dataset

In [4]:
params = {
    'dim': config.IMG_DIM,
    'batch_size': 1,
    'patches_per_image': 1,
    'density_map_multiplication_factor': config.DENSITY_MAP_MULTIPLICATION_FACTOR,
    'shuffle': False,
    'data_augmentation': False,
    'output_mode': 'seg_reg',
    'gt_seg_thr': config.GT_SEG_THR
}

In [5]:
train_generator = DataGenerator(config.DATASET_PATH, 'train', **params)
val_generator = DataGenerator(config.DATASET_PATH, 'val', **params)
test_generator = DataGenerator(config.DATASET_PATH, 'test', **params)

In [6]:
train_gt_counts = load_gt_counts(config.TRAIN_GT_COUNT_PATH)
val_gt_counts = load_gt_counts(config.VAL_GT_COUNT_PATH)
test_gt_counts = load_gt_counts(config.TEST_GT_COUNT_PATH)

### Select the checkpoint file that you want to test/evaluate

In [7]:
checkpoint_filenames = sorted(os.listdir(config.CHECKPOINTS_PATH))
print(checkpoint_filenames)

['best_model.hdf5', 'model.01-0.118.hdf5', 'model.02-0.106.hdf5', 'model.03-0.105.hdf5', 'model.04-0.101.hdf5', 'model.05-0.100.hdf5', 'model.06-0.100.hdf5', 'model.07-0.099.hdf5', 'model.08-0.099.hdf5', 'model.09-0.098.hdf5', 'model.10-0.098.hdf5', 'model.11-0.099.hdf5', 'model.12-0.097.hdf5', 'model.13-0.098.hdf5', 'model.14-0.097.hdf5', 'model.15-0.097.hdf5', 'model.16-0.101.hdf5', 'model.17-0.096.hdf5', 'model.18-0.102.hdf5', 'model.19-0.098.hdf5', 'model.20-0.096.hdf5', 'model.21-0.096.hdf5', 'model.22-0.097.hdf5', 'model.23-0.097.hdf5', 'model.24-0.098.hdf5', 'model.25-0.095.hdf5', 'model.26-0.094.hdf5', 'model.27-0.097.hdf5', 'model.28-0.096.hdf5', 'model.29-0.095.hdf5', 'model.30-0.095.hdf5', 'model.31-0.096.hdf5', 'model.32-0.095.hdf5', 'model.33-0.097.hdf5', 'model.34-0.096.hdf5', 'model.35-0.102.hdf5', 'model.36-0.094.hdf5', 'model.37-0.094.hdf5', 'model.38-0.099.hdf5', 'model.39-0.095.hdf5', 'model.40-0.096.hdf5', 'model.41-0.097.hdf5', 'model.42-0.095.hdf5', 'model.43-0.09

In [9]:
for checkpoint_idx in [50]:
    selected_checkpoint_filename = checkpoint_filenames[checkpoint_idx]
    print(f'selected checkpoint_filename: {selected_checkpoint_filename}')
    
    epoch = selected_checkpoint_filename.split('.')[1].split('-')[0]
    print('epoch:', epoch)
    
    # Set epoch and val loss
    CHECKPOINT_FILENAME = f'{config.CHECKPOINTS_PATH}/{selected_checkpoint_filename}'
    QUANTITATIVE_RESULTS_PATH = f'./{config.SUB_EXPERIMENT_NAME}/results/quantitative/epoch_{epoch}'
    
    !rm -rf $QUANTITATIVE_RESULTS_PATH
    os.makedirs(QUANTITATIVE_RESULTS_PATH)
    
    ## 3. Load the best model
    model = SegRegNet(pretrained_weights=CHECKPOINT_FILENAME)

    train_pred_counts = predict_density_maps_and_get_counts(model, train_generator,
                                                            config.DENSITY_MAP_MULTIPLICATION_FACTOR,
                                                            output_mode='seg_reg')
    val_pred_counts = predict_density_maps_and_get_counts(model, val_generator,
                                                          config.DENSITY_MAP_MULTIPLICATION_FACTOR,
                                                          output_mode='seg_reg')
    test_pred_counts = predict_density_maps_and_get_counts(model, test_generator,
                                                           config.DENSITY_MAP_MULTIPLICATION_FACTOR,
                                                           output_mode='seg_reg')
    
    ## 4. Predict and evaluate
    train_results = evaluation_results_as_dict(train_gt_counts, train_pred_counts, 'train')
    val_results = evaluation_results_as_dict(val_gt_counts, val_pred_counts, 'val')
    test_results = evaluation_results_as_dict(test_gt_counts, test_pred_counts, 'test')

    df = evaluation_results_as_df(train_results, val_results, test_results,
                                  config.ARCHITECTURE_NAME,
                                  config.SUB_EXPERIMENT_NAME,
                                  config.DATASET_NAME)

    df.to_csv(f'{QUANTITATIVE_RESULTS_PATH}/results.csv', index=True)

selected checkpoint_filename: model.50-0.095.hdf5
epoch: 50
